#### MNIST handwritten digits  

The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).
![title](../img/mnist.png)

In [2]:
import tensorflow as tf

# import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
# parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
log_print = 1

# tf graph input (None,784 because it depends on the number of inputs; placeholders are inputted by users, variables in 
# process of execution are changed)
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784; None means, axis=1 is default
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes
W = tf.Variable(tf.ones([784,10]))
b = tf.Variable(tf.ones([10]))

# model (softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), axis)) (softmax(logits))
y_hat = tf.nn.softmax(tf.matmul(x,W) + b)
# */+/- these are scalar multiplications, no axis needed, for vector it will be tf.operation and axis needs to be fixed
# beforehand
# loss 
loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_hat),axis=1))
# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
# initialize
init = tf.global_variables_initializer()

In [10]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        no_of_batches = int(mnist.train.num_examples/batch_size)
        for batch in range(no_of_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _,c = sess.run([optimizer, loss], feed_dict={x:batch_xs, y:batch_ys})
            avg_cost += c / no_of_batches
        if (epoch + 1) % log_print == 0:
            print("Epoch:", '%04d' % (epoch+1), "| loss=", "{:.9f}".format(avg_cost))
            
    print("Optimization Finished!")
    # along the axis of columns which index is highest in terms of value and finally if that matches
    correct_preds = tf.equal(tf.argmax(y_hat,1),tf.argmax(y,1))
    # case true/false to 0/1 and sum_mean
    accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
#     print("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    accuracy_value = sess.run(accuracy, feed_dict={x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})
    print("Accuracy:",accuracy_value)

Epoch: 0001 | loss= 1.183808884
Epoch: 0002 | loss= 0.665408830
Epoch: 0003 | loss= 0.552996026
Epoch: 0004 | loss= 0.498742459
Epoch: 0005 | loss= 0.465566743
Epoch: 0006 | loss= 0.442605241
Epoch: 0007 | loss= 0.425556291
Epoch: 0008 | loss= 0.412229003
Epoch: 0009 | loss= 0.401405380
Epoch: 0010 | loss= 0.392375708
Epoch: 0011 | loss= 0.384811149
Epoch: 0012 | loss= 0.378217488
Epoch: 0013 | loss= 0.372418442
Epoch: 0014 | loss= 0.367335863
Epoch: 0015 | loss= 0.362769650
Epoch: 0016 | loss= 0.358596652
Epoch: 0017 | loss= 0.354906417
Epoch: 0018 | loss= 0.351463047
Epoch: 0019 | loss= 0.348330138
Epoch: 0020 | loss= 0.345415566
Epoch: 0021 | loss= 0.342789008
Epoch: 0022 | loss= 0.340259088
Epoch: 0023 | loss= 0.337925370
Epoch: 0024 | loss= 0.335735542
Epoch: 0025 | loss= 0.333735899
Optimization Finished!
Accuracy: 0.889


#### Learnings:
1. Why use variables?  
   Because sess.run runs a graph from scratch, to cache the variables you have to use tf.Variable   func.eval/sess.run(func) are same